In [213]:
!pip3 install transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [373]:
import sys
sys.path.append('src/')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch
import ast

# panda view settings
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', None)  # Display all rows
pd.set_option('display.max_colwidth', 500)  # Display full text in columns

In [285]:
DEVICE = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

print(DEVICE)

cpu


## Read from exisiting csvs

In [215]:
df = pd.read_csv("./data/toxicbias_train.csv")
df_bias = df[df['bias'] == 'bias']
df_neutral = df[df['bias'] == 'neutral']


In [505]:
# Read the CSV files into DataFrames
df_political_results = pd.read_csv('./results/final/political_results.csv')
df_gender_results = pd.read_csv('./results/final/gender_results.csv')
df_lgbtq_results = pd.read_csv('./results/final/lgbtq_results.csv')
df_race_results = pd.read_csv('./results/final/race_results.csv')
df_religion_results = pd.read_csv('./results/final/religion_results.csv')

df_political_results.drop('is_label_correct', axis=1, inplace=True)
df_gender_results.drop('is_label_correct', axis=1, inplace=True)
df_lgbtq_results.drop('is_label_correct', axis=1, inplace=True)
df_race_results.drop('is_label_correct', axis=1, inplace=True)
df_religion_results.drop('is_label_correct', axis=1, inplace=True)

# Combine the DataFrames into one
combined_df = pd.concat([df_political_results, df_gender_results, df_lgbtq_results, df_race_results, df_religion_results], ignore_index=True)

## Split up dataframe by category

In [216]:
# store unique categories
unique_categories = set()

# Iterate through each entry in the 'category' column
for categories in df['category'].str.split(','):
    for category in categories:
        stripped_category = category.strip()
        if stripped_category and stripped_category.lower() != 'none':
            unique_categories.add(stripped_category)

unique_categories_list = sorted(list(unique_categories))

In [217]:
categorical_dfs = {category: pd.DataFrame(columns=df.columns) for category in unique_categories_list}

# Split categories and add rows to the corresponding dataframes in dictionary
def split_categories_and_add_rows(row):
    categories = row['category'].split(',')
    for category in categories:
        category = category.strip()  # Remove leading/trailing spaces
        if category in categorical_dfs:
            categorical_dfs[category] = categorical_dfs[category].append(row, ignore_index=True)

df_bias.apply(split_categories_and_add_rows, axis=1)

# Print the shape of each category dataframe
for category, category_df in categorical_dfs.items():
    print(f"Dataframe for {category}:")
    print(f"Shape: {category_df.shape}")
    print("\n")

Dataframe for gender:
Shape: (293, 6)


Dataframe for lgbtq:
Shape: (453, 6)


Dataframe for political:
Shape: (172, 6)


Dataframe for race:
Shape: (1768, 6)


Dataframe for religion:
Shape: (1257, 6)




## Split training and test data

In [218]:
training_data = {}
test_data = {}

# Perform the 80-20 split for each category dataframe
for category, df in categorical_dfs.items():
    train_df, test_df = train_test_split(df, test_size=0.8, random_state=42)
    
    training_data[category] = train_df
    test_data[category] = test_df

In [289]:
for category, train_df in training_data.items():
    print('Training data')
    print(f"Dataframe for {category}:")
    print(f"Shape: {train_df.shape}")
    print("\n")

Training data
Dataframe for gender:
Shape: (58, 6)


Training data
Dataframe for lgbtq:
Shape: (90, 6)


Training data
Dataframe for political:
Shape: (34, 6)


Training data
Dataframe for race:
Shape: (353, 6)


Training data
Dataframe for religion:
Shape: (251, 6)




In [288]:
for category, test_df in test_data.items():
    print('Test data')
    print(f"Dataframe for {category}:")
    print(f"Shape: {test_df.shape}")
    print("\n")

Training data
Dataframe for gender:
Shape: (470, 6)


Training data
Dataframe for lgbtq:
Shape: (726, 6)


Training data
Dataframe for political:
Shape: (276, 6)


Training data
Dataframe for race:
Shape: (2282, 6)


Training data
Dataframe for religion:
Shape: (1873, 6)




## Hypotheses

In [548]:
hypotheses = {
  'gender' : [
    'A person belonging to a gender lacks intelligence.',
    'A person belonging to a gender is a criminal.',
    'A person belonging to a gender is violent.',
    'Violence is encouraged against a gender.',
    'A person belonging to a gender is abusive.',
    'A person belonging to a gender should be blamed.',
    'Women make decisions based on emotions.',
    'A gender is unable to do some jobs.',
    'A gender is unable to do something as well as another gender.',
    'A gender should not talk.',
    'A person belonging to a gender is compared to an object.',
    'A gender should be blamed.',
    'A person belonging to a gender has an advantage compared to others.',
    'A person belonging to a gender is misogynist.',
    'A person belonging to a gender is disliked.',
  ],
  'lgbtq' : [
    'Lgbtq persons are believed to have a condition.',
    'Lgbtq should not be part of society.',
    'Violence is encouraged against lgbtq persons.',
    'Lgbtq persons are compared to animals.',
    'Lgbtq persons are believed to have an agenda.',
    'Lgbtq persons should be excluded.',
    'Lgbtq persons should stay home.',
    'Lgbtq persons commit crimes.',
    'Lgbtq persons get preferred to other persons.',
    'Lgbtq persons are seen as not being normal.', 
    'Lgbtq persons should not be allowed to marry.', 
    'Lgbtq persons have a disease.',
    'Lgbtq persons are seen as being disgusting.',
    'Lgbtq persons are not tolerated or desired.',
    'Lgbtq persons are treated differently.',

  ],
  'political' : [ 
      'A person belonging to a political group is stupid.',
      'A person belonging to a political group is crazy.',
      'Things are worse due to a political group.',
      'Members of a political group are lgbtq.',
      'Members of a political group are hypocritical.',
      'Members of a political group dislike a gender or lgbtq.',
      'Members of a political group commit crimes.',
      'Members of a political group are violent.',
      'Members of a political group are hateful.',
      'Members of a political group are compared to Nazis.',
      'Members of a political group are racist.',
      'A person belonging to a political group has a disease.',
      'Members of a political group are taking control.',
      'Members of a political group prefer minorities.',
      'Members of a political group hate their country or race.',
  ],
  'race' : [
    'Members of a race commit crimes.',
    'Members of a race are violent.',
    'Members of a race are compared to animals.',
    'The use of violence is encouraged or desired against a race.',
    'Members of a race should remain in their respective countries.',
    'The integration of a race should be avoided in a country or society.',
    'Members of a race are a threat to others.',
    'A race is inferior when compared to another race.',
    'A race is preferred to another.',
    'Members of a race are stupid.',
    'There are too many members of a race.',
    'Members of a race should be excluded.',
    'Members of a race should go back to their homes.',
    'Members of a race are refused.',
    'Members of a race are racist.',
  ],
  'religion' : [
    'The use of violence is encouraged or desired against a religion.',
    'A religion should be discriminated against.',
    'Members of a religion should be excluded and marginalized.',
    'Members of a religion should remain in their respective countries.',
    'Members of a religion are compared to animals.',
    'A religion or members of a religion are compared to devils or demons.',
    'The integration of a religion should be avoided in a country or society.',
    'A religion or members of a religion are stupid.',
    'A religion is inferior when compared to another religion.',
    'A religion is preferred to another.',
    'Religious practices of a particular faith should be restricted or banned in public spaces.',
    'Religious attire or symbols should be prohibited.',
    'A religion promotes violence or intolerance.',
    'Members of a religion are a threat.',
    'Members of a religion are not trustworthy or loyal.',
  ],
}

## Add neutral cases

In [221]:
def add_rows_to_dataframes(dataframes_dict, new_dataframe):

    for key, df in dataframes_dict.items():
        # Calculate the maximum number of rows to add
        max_rows_to_add = min(len(new_dataframe), len(df))

        # Choose random rows from the new dataframe
        rows_to_add = new_dataframe.sample(n=max_rows_to_add, replace=True)

        rows_to_add = new_dataframe.iloc[:max_rows_to_add]
        df = pd.concat([df, rows_to_add], ignore_index=True)
        dataframes_dict[key] = df


In [223]:
add_rows_to_dataframes(test_data, df_neutral)

## Bart model

In [286]:
def convert_probabilities_batched(probabilities, label_mapping):
    probabilities_list = probabilities.tolist()  # Convert the tensor to a list
    return [{name: round(pred * 100, 1) for pred, name in zip(preds, label_mapping)} for preds in probabilities_list]

In [542]:
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

def bart_nli_batched_hypotheses(premises, hypotheses):
    model_name = 'facebook/bart-large-mnli'
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    model.to(DEVICE) 

    num_premises = len(premises)
    print(num_premises)

    num_hypotheses = len(hypotheses)
    print(num_hypotheses)
    results = {}

    with torch.no_grad():
        for i in range(num_premises):
            labels = []

            # Tokenize the input pair
            inputs = tokenizer([premises[i]] * num_hypotheses, hypotheses, return_tensors='pt', padding=True, truncation=True)

            # Run the input through the model
            logits = model(**inputs.to(DEVICE)).logits

            # Get probabilities and labels for the input
            probabilities = torch.softmax(logits, dim=1)
            label_mapping = ['contradiction', 'neutral', 'entailment']
            labels = [label_mapping[p.argmax().item()] for p in probabilities]

            results[premises[i]] = (labels, convert_probabilities_batched(probabilities, label_mapping))
    return results


## Run data through model, add labels and probabilities

In [502]:
def run_bart_model_add_labels(category):
    # Create a copy of the input dataframe
    df_copy = test_data[category].copy()

    # Run BART NLI on the dataframes for the specified category
    results = bart_nli_batched_hypotheses(list(df_copy['comment_text']), hypotheses[category])

    # Process the results separately and add them to the dataframe
    labels_column = []
    probabilities_column = []

    for premise in df_copy['comment_text']:
        if premise in results:
            labels, probabilities = results[premise]
        else:
            labels, probabilities = None, None
        labels_column.append(labels)
        probabilities_column.append(probabilities)

    df_copy['labels'] = labels_column
    df_copy['probabilities'] = probabilities_column

    return df_copy

In [ ]:
df_results = run_bart_model_add_labels('political')

## Calculate f1 scores

In [536]:
def determine_label_accuracy(df, total_correct_bias=len(hypotheses['race']), total_correct_neutral=len(hypotheses['race'])):
    def is_correct_bias(row, n_labels):
        labels = ast.literal_eval(row['labels'])
        return sum(label == 'entailment' for label in labels) >= n_labels

    def is_correct_neutral(row, n_labels):
        labels = ast.literal_eval(row['labels'])
        print(sum(label != 'entailment' for label in labels))
        print(print(n_labels))
        print(sum(label != 'entailment' for label in labels) >= n_labels)
        return sum(label != 'entailment' for label in labels) >= n_labels

    label_correctness_list = []

    for _, row in df.iterrows():
        # Determine which function to use based on the 'bias' value
        is_correct_fn = is_correct_bias if row['bias'] == 'bias' else is_correct_neutral

        row_results = []

        for n in range(1, total_correct_bias + 1) if row['bias'] == 'bias' else range(1, total_correct_neutral + 1):
            result = is_correct_fn(row, n)
            row_results.append(result)

        label_correctness_list.append(row_results)

    # Add the list as the 'is_label_correct' column in the DataFrame
    df['is_label_correct'] = pd.Series(label_correctness_list)

    df['predicted'] = True
    return df


In [537]:
df_results = determine_label_accuracy(combined_df, 8, 15)

9
1
None
True
9
2
None
True
9
3
None
True
9
4
None
True
9
5
None
True
9
6
None
True
9
7
None
True
9
8
None
True
9
9
None
True
9
10
None
False
9
11
None
False
9
12
None
False
9
13
None
False
9
14
None
False
9
15
None
False
13
1
None
True
13
2
None
True
13
3
None
True
13
4
None
True
13
5
None
True
13
6
None
True
13
7
None
True
13
8
None
True
13
9
None
True
13
10
None
True
13
11
None
True
13
12
None
True
13
13
None
True
13
14
None
False
13
15
None
False
6
1
None
True
6
2
None
True
6
3
None
True
6
4
None
True
6
5
None
True
6
6
None
True
6
7
None
False
6
8
None
False
6
9
None
False
6
10
None
False
6
11
None
False
6
12
None
False
6
13
None
False
6
14
None
False
6
15
None
False
14
1
None
True
14
2
None
True
14
3
None
True
14
4
None
True
14
5
None
True
14
6
None
True
14
7
None
True
14
8
None
True
14
9
None
True
14
10
None
True
14
11
None
True
14
12
None
True
14
13
None
True
14
14
None
True
14
15
None
False
15
1
None
True
15
2
None
True
15
3
None
True
15
4
None
True
15
5
None
True
15
6
None
Tru

In [528]:
df_results.head(1)

,Unnamed: 0,comment_text,identity_attack_x,bias,target,category,rationale,labels,probabilities,predicted,is_label_correct
0,0,Only Republicans should be authorized to conceal carry. Democrats are all mentally ill.,0.272727,bias,democrats,political,prejudice against democrats,"['contradiction', 'entailment', 'neutral', 'contradiction', 'entailment', 'neutral', 'neutral', 'entailment', 'entailment', 'contradiction', 'contradiction', 'entailment', 'entailment', 'entailment', 'contradiction']","[{'contradiction': 65.0, 'neutral': 17.7, 'entailment': 17.3}, {'contradiction': 4.7, 'neutral': 12.1, 'entailment': 83.2}, {'contradiction': 12.9, 'neutral': 83.2, 'entailment': 3.9}, {'contradiction': 76.9, 'neutral': 10.3, 'entailment': 12.8}, {'contradiction': 1.8, 'neutral': 16.8, 'entailment': 81.4}, {'contradiction': 19.8, 'neutral': 46.8, 'entailment': 33.5}, {'contradiction': 18.9, 'neutral': 45.1, 'entailment': 36.0}, {'contradiction': 24.8, 'neutral': 27.1, 'entailment': 48.1}, {'...",True,"[True, True, True, True, True, True, True, False]"


In [529]:
df_results.tail(10)

,Unnamed: 0,comment_text,identity_attack_x,bias,target,category,rationale,labels,probabilities,predicted,is_label_correct
5617,1863,Anarchists and International communists vs National Socialists. SAME SHIT. Only this time Americans who stand with the Constitution are going to wipe the floor of ALL OF THE ANTIFA BLM AND KKKNAZIS.,0.262500,neutral,NaN,none,NaN,"['entailment', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'neutral', 'contradiction', 'contradiction', 'contradiction', 'contradiction']","[{'contradiction': 26.2, 'neutral': 31.3, 'entailment': 42.5}, {'contradiction': 77.0, 'neutral': 22.8, 'entailment': 0.3}, {'contradiction': 84.5, 'neutral': 15.1, 'entailment': 0.4}, {'contradiction': 85.1, 'neutral': 14.2, 'entailment': 0.7}, {'contradiction': 87.6, 'neutral': 8.5, 'entailment': 3.9}, {'contradiction': 95.0, 'neutral': 3.9, 'entailment': 1.1}, {'contradiction': 68.2, 'neutral': 31.5, 'entailment': 0.3}, {'contradiction': 85.9, 'neutral': 13.5, 'entailment': 0.6}, {'contra...",True,"[True, True, True, True, True, True, True, True, True, True, True, True, True, False, False]"
5618,1864,Normal Americans: Hey jackass wheres your tax return?? The Donald: Look over there everybody Scary Muslims,0.657534,neutral,NaN,none,NaN,"['entailment', 'contradiction', 'contradiction', 'contradiction', 'entailment', 'entailment', 'neutral', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'neutral', 'entailment', 'contradiction']","[{'contradiction': 22.5, 'neutral': 27.5, 'entailment': 50.0}, {'contradiction': 95.0, 'neutral': 4.8, 'entailment': 0.3}, {'contradiction': 88.1, 'neutral': 11.7, 'entailment': 0.2}, {'contradiction': 52.5, 'neutral': 47.0, 'entailment': 0.5}, {'contradiction': 13.9, 'neutral': 10.6, 'entailment': 75.5}, {'contradiction': 37.7, 'neutral': 20.6, 'entailment': 41.8}, {'contradiction': 40.0, 'neutral': 59.7, 'entailment': 0.3}, {'contradiction': 78.2, 'neutral': 19.3, 'entailment': 2.4}, {'con...",True,"[True, True, True, True, True, True, True, True, True, True, False, False, False, False, False]"
5619,1865,Who said shes privilege because shes black the KKK?,0.900000,neutral,NaN,none,NaN,"['entailment', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'entailment', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'neutral', 'entailment', 'entailment', 'contradiction']","[{'contradiction': 3.7, 'neutral': 30.3, 'entailment': 66.1}, {'contradiction': 95.2, 'neutral': 4.4, 'entailment': 0.3}, {'contradiction': 92.7, 'neutral': 6.7, 'entailment': 0.5}, {'contradiction': 97.2, 'neutral': 2.6, 'entailment': 0.2}, {'contradiction': 95.5, 'neutral': 2.8, 'entailment': 1.7}, {'contradiction': 16.2, 'neutral': 35.3, 'entailment': 48.5}, {'contradiction': 56.7, 'neutral': 43.0, 'entailment': 0.3}, {'contradiction': 75.0, 'neutral': 22.8, 'entailment': 2.2}, {'contradi...",True,"[True, True, True, True, True, True, True, True, True, True, False, False, False, False, False]"
5620,1866,But its OK when atheists and secularists do it? What gives you the moral or ethical grounds to hate and discriminate against me and my two wives?,0.600000,neutral,NaN,none,NaN,"['neutral', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'neutral', 'contradiction', 'contradiction', 'contradiction', 'contradiction', 'neutral', 'contradiction', 'neutral', 'neutral']","[{'contradiction': 30.0, 'neutral': 60.0, 'entailment': 10.0}, {'contradiction': 89.9, 'neutral': 9.7, 'entailment': 0.5}, {'contradiction': 78.2, 'neutral': 21.2, 'entailment': 0.6}, {'contradiction': 53.5, 'neutral': 45.9, 'entailment': 0.6}, {'contradiction': 93.3, 'neutral': 6.0, 'entailment': 0.7}, {'contradiction': 15.5, 'neutral': 83.4, 'entailment': 1.2}, {'contradiction': 66.3, 'neutral': 33.3, 'entailment': 0.4}, {'contradiction': 84.2, 'neutral': 15.2, 'entailment': 0.6}, {'contra...",True,"[True, True

In [ ]:
# Function to calculate and print F1 score for a given DataFrame
def print_f1_score(df, category_name):
    f1 = f1_score(df['is_label_correct'], df['predicted'])
    print(f"F1 score for {category_name}: {f1}")

In [500]:
def get_combination_f1_scores(df):
  f1_scores = []

  for idx in range(len(df['is_label_correct'].iloc[0])):
      # Get the true labels (ground truth) and predicted labels for the current index
      true_labels = df['is_label_correct'].apply(lambda x: x[idx])

      predicted_labels = df['predicted']

      f1 = f1_score(true_labels, predicted_labels)

      f1_scores.append(f1)
  return f1_scores


In [501]:
get_combination_f1_scores(df_results)

[0.9824807288016818,
 0.8744186046511627,
 0.7515047291487532,
 0.7000895255147717,
 0.6751824817518248,
 0.6715462031107045,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.6666666666666666,
 0.6605166051660516,
 0.6505576208178439,
 0.6118546845124283,
 0.4634920634920635]

In [ ]:
print_f1_score(df_results, 'religion')